## EthSF 24 Test Notebook

# A. Owner

### 1. Create a new session as owner

In [ ]:
from weaveapi.records import *
from weaveapi.options import *
from weaveapi.filter import *
from weaveapi.weaveh import *

import pandas as pd

from web3.auto import w3
from eth_account.messages import encode_defunct
from hexbytes import HexBytes

WEAVE_CONFIG = "config/demo_client_ethan.config"
nodeApiOwner, sessionOwner = connect_weave_api(WEAVE_CONFIG)

organization = "ethan"

data_collection = "vault"
files_table = "files"
wallets_table = "wallets"
hashes_table = "hashes"

### 2. Create a table to store files

In [ ]:
layout = { 
    "columns": { 
        "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
        "ts": { "type": "LONG" },
        "writer": { "type": "STRING" },
        "sig": { "type": "STRING" },
        "access": { "type": "STRING" },
        "did": { "type": "STRING" },
        "name": { "type": "STRING" },
        "recipient": { "type": "STRING" },
        "content": { "type": "STRING" }
    }, 
    "idColumnIndex": 0,  # Autogenerates IDs
    "timestampColumnIndex": 1, # Fills the column automatically with the network time
    "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
    "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
    "allowedRolesColumnIndex": 4, # Fills the column with the allowed reader
    "accessExpiryColumnIndex": 7, # Fills the column with the allowed reader
    "isLocal": False,
    "applyReadTransformations": True
}

nodeApiOwner.dropTable(sessionOwner, data_collection, files_table).get()
reply = nodeApiOwner.createTable(sessionOwner, data_collection, files_table, CreateOptions(False, False, layout)).get()
print(reply)

### 3. Create a table to store weave public keys to wallets associations (and ownership proofs)

In [ ]:
layout = { 
    "columns": { 
        "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
        "ts": { "type": "LONG" },
        "writer": { "type": "STRING" },
        "sig": { "type": "STRING" },
        "wallet": { "type": "STRING" },
        "signature": { "type": "STRING" }
    }, 
    "idColumnIndex": 0,  # Autogenerates IDs
    "timestampColumnIndex": 1, # Fills the column automatically with the network time
    "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
    "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
    "isLocal": False,
    "applyReadTransformations": True
}

nodeApiOwner.dropTable(sessionOwner, data_collection, wallets_table).get()
reply = nodeApiOwner.createTable(sessionOwner, data_collection, wallets_table, CreateOptions(False, False, layout)).get()
print(reply)

### 4. Create a table to store document hashes

- needed because encrypted versions hash are not a guarantee of integrity
- salted hashes could be stored, to avoid dictionary attacks, and the hash made available only on document share
- hashes of the records in this table could be eventually put on a blockchain for higher trust

In [ ]:
layout = { 
    "columns": { 
        "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
        "ts": { "type": "LONG" },
        "writer": { "type": "STRING" },
        "sig": { "type": "STRING" },
        "did": { "type": "STRING" },
        "hash": { "type": "STRING" }
    }, 
    "idColumnIndex": 0,  # Autogenerates IDs
    "timestampColumnIndex": 1, # Fills the column automatically with the network time
    "ownerColumnIndex": 2, # Fills the pubkey column automatically with the public key of the writer
    "signatureColumnIndex": 3, # Fills the column with an EdDSA signature of the record hash
    "isLocal": False,
    "applyReadTransformations": True
}

nodeApiOwner.dropTable(sessionOwner, data_collection, hashes_table).get()
reply = nodeApiOwner.createTable(sessionOwner, data_collection, hashes_table, CreateOptions(False, False, layout)).get()
print(reply)

### 5. Add sample file

In [ ]:
import base64
import uuid

rows = []

access = "skale:0x3389798389CD7C54C02dBB253c9f7e6851672Ab5"
did = "did:ethsf24:" + str(uuid.uuid4()).replace("-", "") 
name = "test"
expiry = "2024-10-20"
content_file = "next.csv"

with open(content_file) as f:
    data = f.read()
    content = base64.b64encode(data.encode()).decode()
    
row = [ None, None, None, None, access, did, name, expiry, content ]
rows.append(row)

records = Records(files_table, rows)
#Superseeded by the UI
#res = nodeApiOwner.write(sessionOwner, data_collection, records, WRITE_DEFAULT).get()
print(res)

### 5. Read files

filter = Filter(None, { "id": "ASC" }, None, [ "did" ], None)
reply = nodeApiOwner.read(sessionOwner, data_collection, files_table, filter, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])

display(df.tail())

display(json.loads(df["sig"][0]))